In [1]:
import os
from tqdm import tqdm

In [2]:
data_dir = 'raw_data'

### XML files

In [15]:
import lxml.etree as ET
import re

In [76]:
full_set = {}

for file in tqdm(os.listdir(data_dir)):
    if file.endswith('.xml'):
        xml = os.path.join(data_dir, file)

        xml_file = ET.parse(xml)

        # Clean up enum and itemize tags
        for enum in (xml_file.findall('.//{req_document.xsd}enum') + xml_file.findall('.//{req_document.xsd}itemize')):
            children = enum.findall('.//{req_document.xsd}item')

            # Make sure that all spaces are gone from both sides of the item
            for child in children:
                if child.text:
                    child.text = child.text.strip()
                    child.text = child.text.strip()

            def adjust_ending_char(sent, char):
                if sent[-1].isalpha():
                    # Add punctuation after sentence when the last character is part of a word
                    return sent + char
                elif sent[-1] == '.' or sent[-1] == ';':
                    # Change the punctuation if there's already something there
                    return sent[:-1] + char
                else:
                    # If any other character (such as quotation marks), add punctuation to it
                    return sent + char

            # Make sure that all items end with a punctuation mark
            if len(children) > 1:
                # End all but the last child in ';'
                for child in children[:-1]:
                    if child.text:
                        child.text = adjust_ending_char(child.text, ';')
                        child.text = adjust_ending_char(child.text, ';')
                
                # End last child in '.'
                if children[-1].text:
                    children[-1].text = adjust_ending_char(children[-1].text, '.')
            else:
                # Only one child: end with '.'
                if children[0].text:
                    children[0].text = adjust_ending_char(children[0].text, '.')
        
        # Get all text_body tags and their stripped contents
        full_body = list(map(lambda x: re.sub(' +', ' ', ''.join(x.itertext()).strip().replace('\n', ' ')), [x for x in xml_file.findall('.//{req_document.xsd}text_body') if x.text is not None]))

        # Make sure there are no "None"s in the body
        full_body = list(filter(lambda x: x != 'None' and bool(x), full_body))

        full_set[file] = full_body


100%|██████████| 80/80 [00:00<00:00, 381.18it/s]
